On va tout d'abord importer les données mesurées pour en déduire les $\rho_i$.

In [1]:
data = """79,05	102,40	115,40	126,10	217,50	240,70
82,4	101,5	114,1	123,1	215,8	239
81,90	104,80	113,20	121,50	214,20	237,50"""

In [30]:
data = data.replace(',', '.')

In [31]:
lines = data.split('\n')

In [32]:
values = [line.split('\t') for line in lines]

In [33]:
values

[['79.05', '102.40', '115.40', '126.10', '217.50', '240.70'],
 ['82.4', '101.5', '114.1', '123.1', '215.8', '239'],
 ['81.90', '104.80', '113.20', '121.50', '214.20', '237.50']]

In [34]:
import numpy as np

In [35]:
import pandas as pd

In [36]:
s = pd.DataFrame(values)

In [37]:
s

,0,1,2,3,4,5
0,79.05,102.40,115.40,126.10,217.50,240.70
1,82.4,101.5,114.1,123.1,215.8,239
2,81.90,104.80,113.20,121.50,214.20,237.50


In [41]:
s.values.astype(np.float)

array([[  79.05,  102.4 ,  115.4 ,  126.1 ,  217.5 ,  240.7 ],
       [  82.4 ,  101.5 ,  114.1 ,  123.1 ,  215.8 ,  239.  ],
       [  81.9 ,  104.8 ,  113.2 ,  121.5 ,  214.2 ,  237.5 ]])

In [42]:
df = pd.DataFrame(s.values.astype(np.float), columns=range(6))

In [44]:
df

,0,1,2,3,4,5
0,79.05,102.4,115.4,126.1,217.5,240.7
1,82.40,101.5,114.1,123.1,215.8,239.0
2,81.90,104.8,113.2,121.5,214.2,237.5


Il nous faut construire le vecteur des $\Delta T _ i$ à partir des mesures de fréquences. On construit d'abord le vecteur des $F_i$

In [45]:
df ** 2

,0,1,2,3,4,5
0,6248.9025,10485.76,13317.16,15901.21,47306.25,57936.49
1,6789.7600,10302.25,13018.81,15153.61,46569.64,57121.00
2,6707.6100,10983.04,12814.24,14762.25,45881.64,56406.25


In [49]:
(df ** 2).ix[0][0]


6248.9024999999992

In [52]:
freqs = np.array([82.4, 110., 146.8, 196., 246.9, 329.6]) # frequencies of the guitar strings, from low E to high E, in Hz
calibration_tensions = np.array([9.59, 11.61, 11.22, 8.43, 8.09, 8.9]) * 9.81 # calibration tensions found on package (in kg) converted to N
mu = calibration_tensions / (4 * 0.648**2 * freqs**2) 
mu

array([ 0.00824942,  0.0056041 ,  0.00304088,  0.00128166,  0.00077511,
        0.00047849])

In [53]:
psi = 4 * 0.648**2 * mu

In [54]:
psi

array([ 0.01385585,  0.00941274,  0.00510752,  0.0021527 ,  0.00130189,
        0.00080368])

In [64]:
T = (df ** 2).values * psi
T

array([[  86.5838593 ,   98.69968579,   68.01761049,   34.23060795,
          61.58768385,   46.56255344],
       [  94.0779    ,   96.97235469,   66.49378303,   32.62124599,
          60.62869632,   45.90715825],
       [  92.93964188,  103.38045091,   65.44893844,   31.77876351,
          59.73299382,   45.33272606]])

**attention** les tensions sont bien en Newton, pas en kg comme dans la feuille Numbers.

On peut maintenant construire une première matrice de 5 lignes fois 6 colonnes.

In [92]:
T_end = T[1, :]
T_start = T[0, :]

mat = np.zeros((5, 6))
dT = T_end - T_start

dT

array([ 7.4940407 , -1.7273311 , -1.52382746, -1.60936196, -0.95898753,
       -0.65539519])

In [93]:
mat

array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

On s'attend à ce qu'il n'y ait qu'une valeur qui a augmenté, ici c'est la première.

In [94]:
np.nonzero(dT > 0)

(array([0]),)

In [95]:
tuned_string = np.nonzero(dT > 0)[0]
assert tuned_string.size == 1
tuned_string = tuned_string[0]

In [96]:
tuned_string

0

In [97]:

cnt = 0
for string in range(6):
        if string == tuned_string:
            continue
        else:
            for other_string in range(6):
                if other_string == tuned_string:
                    mat[cnt, other_string] = 0
                elif other_string == string:
                    mat[cnt, other_string] = dT[tuned_string] + dT[string]
                else:
                    mat[cnt, other_string] = dT[string]
            cnt += 1

In [99]:
mat[0]

array([ 0.       ,  5.7667096, -1.7273311, -1.7273311, -1.7273311,
       -1.7273311])

In [101]:
mat[1]

array([ 0.        , -1.52382746,  5.97021324, -1.52382746, -1.52382746,
       -1.52382746])

In [105]:
mat

array([[ 0.        ,  5.7667096 , -1.7273311 , -1.7273311 , -1.7273311 ,
        -1.7273311 ],
       [ 0.        , -1.52382746,  5.97021324, -1.52382746, -1.52382746,
        -1.52382746],
       [ 0.        , -1.60936196, -1.60936196,  5.88467874, -1.60936196,
        -1.60936196],
       [ 0.        , -0.95898753, -0.95898753, -0.95898753,  6.53505317,
        -0.95898753],
       [ 0.        , -0.65539519, -0.65539519, -0.65539519, -0.65539519,
         6.83864552]])

On peut écrire une fonction avec le code précédent :

In [111]:
def make_matrix(T_end, T_start):
    mat = np.zeros((5, 6))
    dT = T_end - T_start
    tuned_string = np.nonzero(dT > 0)[0]
    assert tuned_string.size == 1
    tuned_string = tuned_string[0]

    cnt = 0
    for string in range(6):
            if string == tuned_string:
                continue
            else:
                for other_string in range(6):
                    if other_string == tuned_string:
                        mat[cnt, other_string] = 0
                    elif other_string == string:
                        mat[cnt, other_string] = dT[tuned_string] + dT[string]
                    else:
                        mat[cnt, other_string] = dT[string]
                cnt += 1

    rhs = -dT[[_ for _ in range(6) if _ != tuned_string]] 
    return mat, rhs

In [112]:
make_matrix(T_end, T_start)

(array([[ 0.        ,  5.7667096 , -1.7273311 , -1.7273311 , -1.7273311 ,
         -1.7273311 ],
        [ 0.        , -1.52382746,  5.97021324, -1.52382746, -1.52382746,
         -1.52382746],
        [ 0.        , -1.60936196, -1.60936196,  5.88467874, -1.60936196,
         -1.60936196],
        [ 0.        , -0.95898753, -0.95898753, -0.95898753,  6.53505317,
         -0.95898753],
        [ 0.        , -0.65539519, -0.65539519, -0.65539519, -0.65539519,
          6.83864552]]),
 array([ 1.7273311 ,  1.52382746,  1.60936196,  0.95898753,  0.65539519]))

On vérifie que le RHS est correct :

In [113]:
dT

array([ 7.4940407 , -1.7273311 , -1.52382746, -1.60936196, -0.95898753,
       -0.65539519])

On peut maintenant construire les deux matrices :

In [114]:
mat1, rhs1 = make_matrix(T[1, :], T[0, :])
mat2, rhs2 = make_matrix(T[2, :], T[1, :])

In [115]:
mat2

array([[ 5.2698381 ,  0.        , -1.13825812, -1.13825812, -1.13825812,
        -1.13825812],
       [-1.04484459,  0.        ,  5.36325164, -1.04484459, -1.04484459,
        -1.04484459],
       [-0.84248247,  0.        , -0.84248247,  5.56561375, -0.84248247,
        -0.84248247],
       [-0.8957025 ,  0.        , -0.8957025 , -0.8957025 ,  5.51239372,
        -0.8957025 ],
       [-0.57443219,  0.        , -0.57443219, -0.57443219, -0.57443219,
         5.83366403]])

In [116]:
rhs2

array([ 1.13825812,  1.04484459,  0.84248247,  0.8957025 ,  0.57443219])

On peut les concaténer :

In [123]:
total_mat = np.vstack((mat1, mat2))
total_rhs = np.vstack((rhs1[:, np.newaxis], 
                       rhs2[:, np.newaxis]))

In [124]:
total_mat

array([[ 0.        ,  5.7667096 , -1.7273311 , -1.7273311 , -1.7273311 ,
        -1.7273311 ],
       [ 0.        , -1.52382746,  5.97021324, -1.52382746, -1.52382746,
        -1.52382746],
       [ 0.        , -1.60936196, -1.60936196,  5.88467874, -1.60936196,
        -1.60936196],
       [ 0.        , -0.95898753, -0.95898753, -0.95898753,  6.53505317,
        -0.95898753],
       [ 0.        , -0.65539519, -0.65539519, -0.65539519, -0.65539519,
         6.83864552],
       [ 5.2698381 ,  0.        , -1.13825812, -1.13825812, -1.13825812,
        -1.13825812],
       [-1.04484459,  0.        ,  5.36325164, -1.04484459, -1.04484459,
        -1.04484459],
       [-0.84248247,  0.        , -0.84248247,  5.56561375, -0.84248247,
        -0.84248247],
       [-0.8957025 ,  0.        , -0.8957025 , -0.8957025 ,  5.51239372,
        -0.8957025 ],
       [-0.57443219,  0.        , -0.57443219, -0.57443219, -0.57443219,
         5.83366403]])

In [125]:
total_rhs

array([[ 1.7273311 ],
       [ 1.52382746],
       [ 1.60936196],
       [ 0.95898753],
       [ 0.65539519],
       [ 1.13825812],
       [ 1.04484459],
       [ 0.84248247],
       [ 0.8957025 ],
       [ 0.57443219]])

On peut maintenant inverser le système :

In [126]:
total_mat.shape

(10, 6)

In [127]:
total_rhs.shape

(10, 1)

In [134]:
rho, err, rank, eigs = np.linalg.lstsq(total_mat, total_rhs)

In [138]:
err

array([ 4.68733407])

In [131]:
rho

array([[ 0.66780388],
       [ 0.77487632],
       [ 0.64784761],
       [ 0.62381181],
       [ 0.447418  ],
       [ 0.2813956 ]])

In [135]:
err

array([ 4.68733407])

On a la solution ! 

**cependant** on note que les raideurs ne sont pas exactement dans l'ordre croissant !! Problème ou pas ?

# Vérification des prédictions du modèle 

On peut maintenant voir si ces coefs permettent de bien prédire la variation de tension observée. On peut tout simplement repartir des matrices mat1 et mat2 et vérifier que l'on tombe bien sur ce qu'on veut.

In [132]:
np.dot(mat1, rho)

array([[ 1.01300744],
       [ 0.62584279],
       [ 0.20832944],
       [ 0.69144332],
       [ 0.28983951]])

In [133]:
rhs1

array([ 1.7273311 ,  1.52382746,  1.60936196,  0.95898753,  0.65539519])

Mouai...

In [136]:
np.dot(mat2, rho)

array([[ 1.24216364],
       [ 1.36353512],
       [ 1.74946958],
       [ 0.47711533],
       [ 0.27046595]])

In [137]:
rhs2

array([ 1.13825812,  1.04484459,  0.84248247,  0.8957025 ,  0.57443219])

Ceci est peut être dû à l'erreur à l'issue de la régression linéaire... on va quand même essayer d'implémenter la méthode d'inversion pour l'accordage.

# Méthode d'accordage 

Il faut ici encore construire la matrice responsable de l'accordage.

In [170]:
tuning_mat = np.zeros((6, 6))

for other_string in range(6):
    for tuning_string in range(6):
        if tuning_string == other_string:
            tuning_mat[other_string, tuning_string] = 1.
        else:
            tuning_mat[other_string, tuning_string] = \
                    psi[tuning_string] / psi[other_string] * \
                        (- rho[other_string] / (1 + np.sum([rho[i] for i in range(6) if i != tuning_string])))

In [171]:
tuning_mat

array([[ 1.        , -0.12367146, -0.06486027, -0.02716512, -0.01570343,
        -0.0093073 ],
       [-0.30212911,  1.        , -0.11078464, -0.04639941, -0.02682225,
        -0.01589734],
       [-0.46552087, -0.32547441,  1.        , -0.07149227, -0.04132775,
        -0.02449463],
       [-1.06351887, -0.74357178, -0.38997086,  1.        , -0.09441648,
        -0.05595991],
       [-1.26128768, -0.88184418, -0.46248868, -0.19370198,  1.        ,
        -0.06636604],
       [-1.28501663, -0.89843455, -0.4711896 , -0.19734615, -0.11408048,
         1.        ]])

In [172]:
np.dot(tuning_mat, np.array([1, 0, 0, 0, 0, 0]))

array([ 1.        , -0.30212911, -0.46552087, -1.06351887, -1.26128768,
       -1.28501663])

On vérifie que ces termes sont les bons.

In [173]:
psi[0] / psi[1] * (- rho[1] / (1 + np.sum([rho[k] for k in range(6) if k != 0])))

array([-0.30212911])

On écrit la fonction pour calculer la matrice :

In [197]:
def compute_tuning_matrix(psi, rho):
    tuning_mat = np.zeros((6, 6))

    for other_string in range(6):
        for tuning_string in range(6):
            if tuning_string == other_string:
                tuning_mat[other_string, tuning_string] = 1.
            else:
                tuning_mat[other_string, tuning_string] = \
                        psi[tuning_string] / psi[other_string] * \
                            (- rho[other_string] / (1 + np.sum([rho[i] for i in range(6) if i != tuning_string])))
    return tuning_mat

In [198]:
compute_tuning_matrix(psi, rho)

array([[ 1.        , -0.12367146, -0.06486027, -0.02716512, -0.01570343,
        -0.0093073 ],
       [-0.30212911,  1.        , -0.11078464, -0.04639941, -0.02682225,
        -0.01589734],
       [-0.46552087, -0.32547441,  1.        , -0.07149227, -0.04132775,
        -0.02449463],
       [-1.06351887, -0.74357178, -0.38997086,  1.        , -0.09441648,
        -0.05595991],
       [-1.26128768, -0.88184418, -0.46248868, -0.19370198,  1.        ,
        -0.06636604],
       [-1.28501663, -0.89843455, -0.4711896 , -0.19734615, -0.11408048,
         1.        ]])

On peut maintenant inverser la matrice en calculant l'accordage cible.

In [174]:
freqs

array([  82.4,  110. ,  146.8,  196. ,  246.9,  329.6])

Le $\Delta f$ à appliquer peut se calculer en faisant la différence à obtenir :

In [184]:
target_freqs = freqs.copy()

In [185]:
current_freqs = df.values[2, :]

L'écart que l'on cherche à obtenir est donc :

In [187]:
target_freqs - current_freqs

array([  0.5,   5.2,  33.6,  74.5,  32.7,  92.1])

On doit le mettre au carré :

In [188]:
target_dF = (target_freqs - current_freqs) ** 2

In [189]:
Delta_F = np.linalg.solve(tuning_mat, target_dF)
Delta_F

array([  1072.41247448,   1801.75701947,   3801.04449148,  11292.69979243,
         9053.27646867,  16531.6242485 ])

On doit prendre la racine de ce grand F pour trouver les fréquences à appliquer :

In [190]:
np.sqrt(Delta_F)

array([  32.74770945,   42.44710849,   61.65261139,  106.26711529,
         95.14870713,  128.57536408])

In [191]:
np.sqrt(np.dot(tuning_mat, Delta_F))

array([  0.5,   5.2,  33.6,  74.5,  32.7,  92.1])

In [192]:
np.sqrt(current_F)

array([  81.9,  104.8,  113.2,  121.5,  214.2,  237.5])

In [194]:
current_freqs + np.sqrt(np.dot(tuning_mat, Delta_F))

array([  82.4,  110. ,  146.8,  196. ,  246.9,  329.6])

Cela devrait marcher. Voyons voir quels deltas de fréquence il faut imposer.

In [195]:
np.sqrt(Delta_F)

array([  32.74770945,   42.44710849,   61.65261139,  106.26711529,
         95.14870713,  128.57536408])

Testons !

In [196]:
current_freqs + np.sqrt(Delta_F)

array([ 114.64770945,  147.24710849,  174.85261139,  227.76711529,
        309.34870713,  366.07536408])

Conclusion : ça n'a pas marché et le pont n'a pas arrêté de monter. Je pense que cela vient d'autre chose : la tension du set de cordes n'était pas adaptée !!! 

Je change donc de set de cordes et je recommence.

# Nouvel essai 

J'ai mis des nouvelles cordes dont les propriétés sont les suivantes :

- mi aigu .009 5.94 kg
- si .011 4.99 kg
- sol .016 6.67 kg
- ré .026 8.34 kg
- la .036 8.84 kg
- mi grave .046 7.94 kg

On en déduit les $\mu_i$ :

In [199]:
freqs = np.array([82.4, 110., 146.8, 196., 246.9, 329.6]) # frequencies of the guitar strings, from low E to high E, in Hz
calibration_tensions = np.array([7.94, 8.84, 8.34, 6.67, 4.99, 5.94]) * 9.81 # calibration tensions found on package (in kg) converted to N
mu = calibration_tensions / (4 * 0.648**2 * freqs**2) 

In [200]:
mu

array([ 0.00683007,  0.00426703,  0.00226034,  0.00101408,  0.0004781 ,
        0.00031935])

J'ai remesuré la longueur des cordes et j'ai trouvé 65 cm.

On en déduit les Psi_i :

In [201]:
psi = 4 * 0.648**2 * mu

In [202]:
psi

array([ 0.01147189,  0.00716698,  0.0037965 ,  0.00170327,  0.00080302,
        0.00053639])

On peut maintenant remesurer les différentes fréquences sur la guitare :

In [204]:
lines = """83,55	94,70	193,7	138,8	203	190
89,2	93,3	192,55	135,2	200,55	186,9
87,8	99,2	191,25	130,9	197,85	183,7""".replace(',', '.').split('\n')

In [207]:
history = np.array([line.split('\t') for line in lines], dtype=np.float)
history

array([[  83.55,   94.7 ,  193.7 ,  138.8 ,  203.  ,  190.  ],
       [  89.2 ,   93.3 ,  192.55,  135.2 ,  200.55,  186.9 ],
       [  87.8 ,   99.2 ,  191.25,  130.9 ,  197.85,  183.7 ]])

In [209]:
T = (history ** 2) * psi

In [210]:
mat1, rhs1 = make_matrix(T[1, :], T[0, :])
mat2, rhs2 = make_matrix(T[2, :], T[1, :])

In [211]:
total_mat = np.vstack((mat1, mat2))
total_rhs = np.vstack((rhs1[:, np.newaxis], 
                       rhs2[:, np.newaxis]))

In [212]:
rho, err, rank, eigs = np.linalg.lstsq(total_mat, total_rhs)

In [213]:
rho

array([[ 0.68364021],
       [ 0.38595602],
       [ 0.35713731],
       [ 0.36234102],
       [ 0.12312326],
       [ 0.07544867]])

In [214]:
err

array([ 15.1809818])

Cette fois-ci on constate que les rho_i sont dans l'ordre !

In [215]:
np.dot(mat1, rho)

array([[ 1.86173921],
       [ 1.79984672],
       [ 1.86626743],
       [ 0.34330066],
       [ 0.02756061]])

In [216]:
rhs1

array([ 1.88634787,  1.68635607,  1.68010244,  0.79394572,  0.62671325])

In [217]:
np.dot(mat2, rho)

array([[ 1.01157584],
       [-0.12690092],
       [-0.17216428],
       [-0.38132147],
       [-0.40471716]])

In [218]:
rhs2

array([ 2.84273508,  1.89422372,  1.94892905,  0.86379456,  0.63611618])

In [219]:
np.dot(total_mat, rho)

array([[ 1.86173921],
       [ 1.79984672],
       [ 1.86626743],
       [ 0.34330066],
       [ 0.02756061],
       [ 1.01157584],
       [-0.12690092],
       [-0.17216428],
       [-0.38132147],
       [-0.40471716]])

In [220]:
total_rhs

array([[ 1.88634787],
       [ 1.68635607],
       [ 1.68010244],
       [ 0.79394572],
       [ 0.62671325],
       [ 2.84273508],
       [ 1.89422372],
       [ 1.94892905],
       [ 0.86379456],
       [ 0.63611618]])

Bizarre la vérification...

In [223]:
tuning_mat = compute_tuning_matrix(psi, rho)

In [224]:
tuning_mat

array([[ 1.        , -0.16416206, -0.08600736, -0.03866298, -0.01670581,
        -0.01097622],
       [-0.2681349 ,  1.        , -0.07772219, -0.03493854, -0.01509652,
        -0.00991887],
       [-0.46838578, -0.25913886,  1.        , -0.06103165, -0.02637104,
        -0.01732656],
       [-1.05922014, -0.58602356, -0.30702796,  1.        , -0.05963618,
        -0.03918276],
       [-0.7634211 , -0.42236995, -0.22128698, -0.09947538,  1.        ,
        -0.02824053],
       [-0.70036106, -0.38748138, -0.20300825, -0.09125852, -0.03943171,
         1.        ]])

In [225]:
target_freqs = freqs.copy()

In [226]:
current_freqs = history[2, :]

In [228]:
target_freqs - current_freqs

array([  -5.4 ,   10.8 ,  -44.45,   65.1 ,   49.05,  145.9 ])

In [227]:
target_dF = (target_freqs - current_freqs) ** 2

In [229]:
Delta_F = np.linalg.solve(tuning_mat, target_dF)
Delta_F

array([  1366.2601893 ,   1472.7920829 ,   4161.43823636,   9189.9578263 ,
         6605.23136645,  24758.28871927])

In [230]:
np.sqrt(Delta_F)

array([  36.96295699,   38.37697334,   64.50921048,   95.86426772,
         81.2725745 ,  157.3476683 ])

In [231]:
for _ in np.sqrt(Delta_F):
    print("{:.2f}".format(_))

36.96
38.38
64.51
95.86
81.27
157.35
